In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
# Pick directory containing Triaxys MESSAGE .log files
global message_directory = pick_folder("C:\\QGHL\\Wave_data\\")

# build list of all message files in selected directory
message_files = filter(x->occursin("_MESSAGE",x), readdir(message_directory));
global message_files = message_files[findall(x->endswith(uppercase(x), ".LOG"), message_files)]

# Check whether any message files exist in selected directory. If not, EXIT!
if length(message_files) == 0
    println("No message files found in "*message_directory)
    exit;
else
    println(string(length(message_files)) * " MESSAGE files found")
end

In [ ]:
function calc_uv(speed, direction)
    v = cos.(deg2rad.(direction)) .* speed
    u = (speed.^2 .- v.^2).^0.5

    neg = direction.>180
    u[neg] = -u[neg]

    return(u,v)
    
    end


function calc_xy(Speed, Direction)
    
    x = []; y = []
    
    for i in 1:length(Speed)

        if Direction[i] < π
        # process bearings less than 180°

            if Direction[i] < π/2
            # process bearings less than 90° 

                push!(x,Speed[i] .* sin.(Direction[i]))
                push!(y,Speed[i] .* cos.(Direction[i]))

            else
            # process bearings greater than 90°  

                push!(x,Speed[i] .* cos.(Direction[i] - π/2))
                push!(y,-Speed[i] .* sin.(Direction[i] - π/2))

            end

        else
        # process bearings greater than 180° 

            if Direction[i] < 3*π/2
            # process bearings less than 270°

                push!(x,-Speed[i] .* sin.(Direction[i] - π))
                push!(y,-Speed[i] .* cos.(Direction[i] - π))

            else
            # process bearings greater than 270°    
                push!(x,-Speed[i] .* cos.(Direction[i] - 3*π/2))
                push!(y,Speed[i] .* sin.(Direction[i] - 3*π/2))

            end

        end

    end
        
    return(x,y)
            
    end    # calc_xy()


function read_adcp_data(infil)

    # read contents of file into comma-separated array
    global data = split.(readlines(infil),",");

    col_names = ["Date", "SST_buoy","SST_adcp","Pressure","Sound","Pitch","Roll","Heading","Cells","Cell_Size","Blanking","Head_depth","Speed", "Direction","X","Y"]
    global adcp_df = DataFrame([ name =>[] for name in col_names])

    for row in eachrow(data)
                
        if (row[1][6] == "4")
    
            date = Dates.DateTime.(row[1][3] .* row[1][4],"yymmddHHMMSS") + Dates.Year(2000);
            sst_buoy = parse.(Float64, row[1][7])
            sst_adcp = parse.(Float64, row[1][8])

            if data[1][5] in aquadopp
                        
                pressure = -99.9
                speed_of_sound = -99.9
                pitch = -99.9
                roll = -99.9
                heading = -99.9
                row[1][9] = replace(row[1][9],"@&7E" => ",")
                cc = split(row[1][9],",")
                cells = parse.(Int, cc[2])
                cell_size = parse.(Float64, cc[3])
                blanking = parse.(Float64, cc[4])
                head_depth = parse.(Float64, cc[5])

                global yy = row[1][10]
                
                row[1][10] = replace(row[1][10],"@&2C" => ",")
                global zz = row[1][10]
                        
                if occursin(",,",zz)
                    temp = replace(zz,",,"=>"0,0,")
                    zz =replace(temp,",,*"=>",0,0*")
                    println(zz)
                end
                        
                global aa = split(split(zz,"*")[1],",")
                global bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
                speed = bb[1,:]
                direction = bb[2,:]
                        
            elseif data[1][5] in signature500
                        
                pressure = parse.(Float64, row[1][9]) * 100
                speed_of_sound = parse.(Float64, row[1][10])
                pitch = parse.(Float64, row[1][11])
                roll = parse.(Float64, row[1][12])
                heading = parse.(Float64, row[1][13])
                row[1][14] = replace(row[1][14],"@&7E" => ",")
                cc = split(row[1][14],",")
                cells = parse.(Int, cc[2])
                cell_size = parse.(Float64, cc[3])
                blanking = parse.(Float64, cc[4])
                head_depth = parse.(Float64, cc[5])
                
                row[1][15] = replace(row[1][15],"@&2C" => ",")
                aa = split(row[1][15],",")
                bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
                speed = bb[1,:]
                direction = bb[2,:]
                        
            end    
                    
            y = cos.(deg2rad.(direction)) .* speed
            x = (speed.^2 .- y.^2).^0.5

            neg = direction.>180
            x[neg] = -x[neg]
                      
            push!(adcp_df,[date,sst_buoy,sst_adcp,pressure,speed_of_sound,pitch,roll,heading,cells,cell_size,blanking,head_depth,speed,direction,x,y])

        end

    end
        
    return(adcp_df)

    end


dates = [string(Dates.Date(split(fil,"_")[1],"yyyymmdd")) for fil in message_files];

# Identify the Triaxys buoy fleet
aquadopp = ["610701ae49aa0034","610101ae49aa0682"]
signature500 = ["610501ae49a9adfb", "610601ae49a9a8f9", "610101ae49a9a344"]

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global date_choice = get_value(lb);
    global file_no = findfirst(occursin.(date_choice, dates))
    
    global msg_file = message_files[file_no]
    println(msg_file * " selected")
    flush(stdout)
    
    global infil = message_directory * "\\" * msg_file
    global adcp_data = []

    col_names = ["Date", "SST_buoy","SST_adcp","Pressure","Sound","Pitch","Roll","Heading","Cells","Cell_Size","Blanking","Head_depth","Speed", "Direction","X","Y"]
    global adcp_df = DataFrame([ name =>[] for name in col_names])

    #open selected file and read its contents

    adcp_df = read_adcp_data(infil)
    adcp_df = sort!(adcp_df, (:Date))

    p1 = plot(adcp_df.Date,adcp_df.SST_buoy, c=:blue, lw=2, label="SST Buoy")
    p1 = plot!(adcp_df.Date,adcp_df.SST_adcp, c=:red, lw=2,label="SST ADCP")
    
    p2 = plot(adcp_df.Date,adcp_df.Pressure, c=:blue, lw=2,label="Pressure (hPa)")
    p2 = plot!(twinx(),adcp_df.Date,adcp_df.Sound, c=:red, lw=2,label="\nSpeed of Sound (hPa)")

 ##   plot_p1 = plot(p1, p2, layout = (2, 1), xlim=(first(adcp_df.Date),last(adcp_df.Date)), size = (1400, 600), framestyle = :box, 
 ##           tick_direction=:out,leftmargin=15Plots.mm, topmargin = 1Plots.mm, fg_legend=:transparent, bg_legend=:transparent, legend=:topleft)

 ##   display(plot_p1)

    blanking = adcp_df.Blanking[1]
    cells = adcp_df.Cells[1]
    cell_size = adcp_df.Cell_Size[1]

    X = 1:length(adcp_df.Date) ##adcp_df.Date #1:length(adcp_df.Date)
    q1 = hline(adcp_df.Date, [0], lw=1, label="")

    for ii in 0:cell_size:cells-1

        ii_int = Int(ii+1)

        speed = hcat(adcp_df.Speed...)[ii_int,:]
        speed[speed.>=5] .= 0
        speed = speed.*5

        direction = hcat(adcp_df.Direction...)[ii_int,:]

        Y = zeros(length(speed)) .- (blanking - ii_int)
##        u,v = calc_uv(speed, direction)
        u, v = calc_xy(speed, direction)
        q1 = quiver!(X,-Y,quiver=(u,v), arrow=4, la=1)

    end

    for i in 5:5:20
        q1 = hline!(X, [-i], lw=1, color=:grey, label="")
    end

    # display plots to screen
    tm_tick = range(first(adcp_df.Date),last(adcp_df.Date),step=Hour(1))
    ticks = Dates.format.(tm_tick,"HH:MM")

    plot_q = plot(q1, xticks=(tm_tick,ticks), xlims=(first(X),last(X)), ylims= (-20,1), size= (1500, 800), framestyle= :box, 
        tick_direction=:out,topmargin = 1Plots.mm)

    display(plot_q)
    

end

In [ ]:
X = adcp_df.Speed[1]; Y = adcp_df.Direction[1]
u,v = calc_xy(X,Y)

In [ ]:
quiver(X,-Y,quiver=(u,v), arrow=4, la=1)

In [ ]:
blanking = adcp_df.Blanking[1]
cells = adcp_df.Cells[1]
cell_size = adcp_df.Cell_Size[1]

X = adcp_df.Date #1:length(adcp_df.Date)
q1 = hline(adcp_df.Date, [0], lw=1, label="")

for ii in 0:cell_size:cells-1

    ii_int = Int(ii+1)

    speed = hcat(adcp_df.Speed...)[ii_int,:]
    speed[speed.>=5] .= 0

    direction = hcat(adcp_df.Direction...)[ii_int,:]

    Y = zeros(length(speed)) .- (blanking - ii_int)
    u,v = calc_uv(speed, direction)
    q1 = quiver!(X,-Y,quiver=(u,v), arrow=4, la=1)

end

for i in 5:5:20
    q1 = hline!(X, [-i], lw=1, color=:grey, label="")
end

# display plots to screen
tm_tick = range(first(adcp_df.Date),last(adcp_df.Date),step=Hour(1))
ticks = Dates.format.(tm_tick,"HH:MM")

plot_q = plot(q1, xticks=(tm_tick,ticks), xlims=(first(X),last(X)), ylims= (-20,1), size= (1500, 800), framestyle= :box, 
    tick_direction=:out,topmargin = 1Plots.mm)

display(plot_q)


In [ ]:
function read_adcp_data(msg_file)
########################################     
    adcp_data = []

    infil = message_directory * "\\" * msg_file

    open(infil) do io

        global i = 0
        data = split(readline(io),",")

        if (data[6] == "4")
            i += 1
#            println(data)
            push!(adcp_data,data)
        end


    close(io)
    end
    
    return(adcp_data)
    
end


function plot_quiver(adcp_data)
######################################## 

    # convert sensor data string into speed and direction arrays
    arry = replace(adcp_data[1][15],"@&2C" => ",")
    arry = split(arry,",")
    arry = reshape(parse.(Float64,arry),2,Int(length(arry)/2))
    
    speed = arry[1,:]
    direction = arry[2,:]

    X = 1:length(speed)
    Y = zeros(length(speed))

    y = cos.(deg2rad.(direction)) .* speed
    x = (speed.^2 .- y.^2).^0.5
    
    neg = direction.>180
    x[neg] = -x[neg]

"""
    for i in 1:length(x)
        @printf("%8.1f %8.1f %8.1f %8.1f \n", speed[i],direction[i],x[i],y[i]) 
        flush(stdout)
    end
"""
    
    q = quiver(X,Y,quiver=(x,y),aspect_ratio=:equal)

    plot_q = plot(q, aspect_ratio=:equal, xlim=[0,Inf], ylim=(-1,1), size = (1400, 400), framestyle = :box, 
        tick_direction=:out,topmargin = 1Plots.mm)

    display(plot_q)

    return()
    
    end


dates = [string(Dates.Date(split(fil,"_")[1],"yyyymmdd")) for fil in message_files];

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global date_choice = get_value(lb);
    global file_no = findfirst(occursin.(date_choice, dates))
    
    global msg_file = message_files[file_no]
    println(msg_file * " selected")
    flush(stdout)
    
    global adcp_data = read_adcp_data(msg_file)
    plot_quiver(adcp_data)

end



In [ ]:
adcp_data = []

infil = message_directory * "\\" * msg_file

open(infil) do io

    global i = 0
    data = split(readline(io),",")

    if (data[i+1][6] == "4")
        i += 1
        println(data)
        push!(adcp_data,data)
    end


close(io)
end


In [ ]:
open(infil) do io
    split(readline(io),",")
close(io)

In [ ]:
data

In [ ]:
# convery string vector into array
adcp_array = permutedims(reshape(hcat(adcp_data...), (length(adcp_data[1]), length(adcp_data))));

# get dates
global dates = Dates.DateTime.(adcp_array[:,3] .* adcp_array[:,4],"yymmddHHMMSS");

# create array of strings containing speed and direction
arry = replace.(adcp_array[:,15],"@&2C" => ",")

# extract speed and direction from array
speed = []
direction = []
global x = []
global y = []
for line in arry
    aa = split(line,",")
    bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
    push!(speed,bb[1,:])
    push!(direction,bb[2,:])
    
    # push calculated x and y values of current speed and direction to arrays
    yy = cos.(deg2rad.(bb[2,:])) .* bb[1,:]
    xx = (bb[1,:].^2 .- yy.^2).^0.5

    # negate x values in quadrants where bearing > 180 degrees
    neg = bb[2,:].>180
    xx = -xx[neg]
    
    push!(y,yy)
    push!(x,xx)
    
end    

In [ ]:
adcp_data = []

infil = message_directory * "\\" * msg_file

open(infil) do io

    global i = 0
    global data = split(readline(io),",")
    println(data)

    if (data[6] == "4")
        i += 1
#        println(data)
        push!(adcp_data,data)
    end


close(io)
end

In [ ]:
col_names = ["Date", "SST_buoy","SST_adcp","Speed", "Direction"]
adcp_df = DataFrame([ name =>[] for name in col_names])

#open selected file and read its contents
open(infil) do io

    while !eof(io)
        
        data = readline(io)
        data = split(data,",")
        if (data[6] == "4")
            
            date = Dates.DateTime.(data[3] .* data[4],"yymmddHHMMSS");
            sst_buoy = parse.(Float64, data[7])
            sst_adcp = parse.(Float64, data[8])
            data = replace(data[15],"@&2C" => ",")
            aa = split(data,",")
            bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
            speed = bb[1,:]
            direction = bb[2,:]
#            println(date,' ',speed,' ',direction)
            push!(adcp_df,[date,sst_buoy,sst_adcp,speed,direction])
        end
        
    end

end

In [ ]:
arry = replace(adcp_data[:,15],"@&7E" => ",")
arry = split(arry,",")
arry = reshape(parse.(Float64,arry),2,Int(length(arry)/2))

speed = arry[1,:]
direction = arry[2,:]

X = 1:length(speed)
Y = zeros(length(speed))

y = cos.(deg2rad.(direction)) .* speed
x = (speed.^2 .- y.^2).^0.5

neg = direction.>180
x[neg] = -x[neg]


for i in 1:length(x)
    @printf("%8.1f %8.1f %8.1f %8.1f \n", speed[i],direction[i],x[i],y[i]) 
    flush(stdout)
end

In [ ]:
adcp_df = sort!(adcp_df, (:Date))

p1 = plot(adcp_df.Date,adcp_df.SST_buoy, c=:blue, lw=2, label="SST Buoy")
p1 = plot!(adcp_df.Date,adcp_df.SST_adcp, c=:red, lw=2,label="SST ADCP")

plot_p1 = plot(p1, xlim=(first(adcp_df.Date),last(adcp_df.Date)), size = (1400, 600), framestyle = :box, 
        tick_direction=:out,leftmargin=15Plots.mm, topmargin = 1Plots.mm, fg_legend=:transparent, bg_legend=:transparent, legend=:topleft)

display(plot_p1)

In [ ]:
"""
function meshgrid(x, y)
    X = [i for i in x, j in 1:length(y)]
    Y = [j for i in 1:length(x), j in y]
    return X, Y
end
"""

meshgrid(x, y) = (repeat(x, outer=length(y)), repeat(y, inner=length(x)))

X,Y = meshgrid(adcp_df.Date, 1:1:20)
speed = adcp_df.Speed
direction = adcp_df.Direction

x = adcp_df.X
y = adcp_df.Y

"""
for i in 1:length(x)
    @printf("%8.1f %8.1f %8.1f %8.1f \n", speed[i],direction[i],x[i],y[i]) 
    flush(stdout)
end
"""

q = quiver(X,Y,quiver=(x,y),aspect_ratio=:equal)

plot_q = plot(q, aspect_ratio=:equal, xlim=[0,Inf], ylim=(-1,1), size = (1400, 400), framestyle = :box, 
    tick_direction=:out,topmargin = 1Plots.mm)

display(plot_q)

In [ ]:
# Identify the Triaxys buoy fleet
aquadopp = ["610701ae49aa0034","610101ae49aa0682"]
signature500 = ["610501ae49a9adfb", "610601ae49a9a8f9", "610101ae49a9a344"]

infil = message_directory * "\\" * msg_file

    open(infil) do io

        global i = 0
        global data = split(readline(io),",")

        close(io)

    end

In [ ]:
# Identify the Triaxys buoy fleet
aquadopp = ["610701ae49aa0034","610101ae49aa0682"]
signature500 = ["610501ae49a9adfb", "610601ae49a9a8f9", "610101ae49a9a344"]

# build file name including full path
infil = message_directory * "\\" * msg_file

# read contents of file into comma-separated array
data = split.(readlines(infil),",");

# determine whether data is from Aquadopp or Signature 500 
signature = data[1][5]
if signature in aquadopp
    println("Aquadopp")
elseif signature in signature500
    println("Signature 500")
else
    println("Buoy signature not recognised")
end

In [ ]:
# build file name including full path
infil = message_directory * "\\" * msg_file

function read_adcp_data(infil)
    
    # Identify the Triaxys buoy fleet
    aquadopp = ["610701ae49aa0034","610101ae49aa0682"]
    signature500 = ["610501ae49a9adfb", "610601ae49a9a8f9", "610101ae49a9a344"]


    # read contents of file into comma-separated array
    global data = split.(readlines(infil),",");

    col_names = ["Date", "SST_buoy","SST_adcp","Pressure","Sound","Pitch","Roll","Heading","Cells","Cell_Size","Blanking","Head_depth","Speed", "Direction","X","Y"]
    global adcp_df = DataFrame([ name =>[] for name in col_names])

    for row in eachrow(data)
        if (row[1][6] == "4")
    
            date = Dates.DateTime.(row[1][3] .* row[1][4],"yymmddHHMMSS") + Dates.Year(2000);
            sst_buoy = parse.(Float64, row[1][7])
            sst_adcp = parse.(Float64, row[1][8])

            if data[1][5] in aquadopp
##                println("Aquadopp")
                pressure = -99.9
                speed_of_sound = -99.9
                pitch = -99.9
                roll = -99.9
                heading = -99.9
                row[1][9] = replace(row[1][9],"@&7E" => ",")
                cc = split(row[1][9],",")
                cells = parse.(Int, cc[2])
                cell_size = parse.(Float64, cc[3])
                blanking = parse.(Float64, cc[4])
                head_depth = parse.(Float64, cc[5])
                
                global yy = row[1][10]
                row[1][10] = replace(row[1][10],"@&2C" => ",")
                global zz = row[1][10]
                if occursin(",,",zz)
                    temp = replace(zz,",,"=>"0,0,")
                    zz =replace(temp,",,*"=>"*")
                end
                global aa = split(split(zz,"*")[1],",")

                global bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
                speed = bb[1,:]
                direction = bb[2,:]
            elseif data[1][5] in signature500
                println("Signature 500")
                pressure = parse.(Float64, row[1][9]) * 100
                speed_of_sound = parse.(Float64, row[1][10])
                pitch = parse.(Float64, row[1][11])
                roll = parse.(Float64, row[1][12])
                heading = parse.(Float64, row[1][13])
                row[1][14] = replace(row[1][14],"@&7E" => ",")
                cc = split(row[1][14],",")
                cells = parse.(Int, cc[2])
                cell_size = parse.(Float64, cc[3])
                blanking = parse.(Float64, cc[4])
                head_depth = parse.(Float64, cc[5])
                
                row[1][15] = replace(row[1][15],"@&2C" => ",")
                aa = split(row[1][15],",")
                bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
                speed = bb[1,:]
                direction = bb[2,:]
            else
                println("Do not recognise buoy signature")
            end    
            y = cos.(deg2rad.(direction)) .* speed
            x = (speed.^2 .- y.^2).^0.5

            neg = direction.>180
            x[neg] = -x[neg]
            
            
            push!(adcp_df,[date,sst_buoy,sst_adcp,pressure,speed_of_sound,pitch,roll,heading,cells,cell_size,blanking,head_depth,speed,direction,x,y])

        end

    end

    return(adcp_df)

end




In [ ]:
infil = message_directory * "\\" * msg_file

read_adcp_data(infil)

In [ ]:
for row in eachrow(data)
    if (row[1][6] == "4")

        date = Dates.DateTime.(row[1][3] .* row[1][4],"yymmddHHMMSS") + Dates.Year(2000);
        sst_buoy = parse.(Float64, row[1][7])
        sst_adcp = parse.(Float64, row[1][8])

        if data[1][5] in aquadopp
            pressure = -99.9
            speed_of_sound = -99.9
            pitch = -99.9
            roll = -99.9
            heading = -99.9
            row[1][9] = replace(row[1][9],"@&7E" => ",")
            cc = split(row[1][9],",")
            cells = parse.(Int, cc[2])
            cell_size = parse.(Float64, cc[3])
            blanking = parse.(Float64, cc[4])
            head_depth = parse.(Float64, cc[5])
            
            global yy = row[1][10]
            row[1][10] = replace(row[1][10],"@&2C" => ",")
            global zz = row[1][10]
            if occursin(",,",zz)
                temp = replace(zz,",,"=>"0,0,")
                zz =replace(temp,",,*"=>"*")
            end
            global aa = split(split(zz,"*")[1],",")
            global bb = reshape(parse.(Float64,aa),2,Int(length(aa)/2))
            speed = bb[1,:]
            direction = bb[2,:]

        end

    end

end

In [ ]:
zz

In [ ]:
if occursin(",,",zz)
    temp = replace(zz,",,"=>"0,0,")
    replace(temp,",,*"=>"*")
end